# Exercise 4

## Question 1
Create tables to keep track of trading positions

**Hint**

After each trade, we want to keep at least the following information of each traded instrument:
1. the date and time when the trade takes place
2. the name of the instrument traded
3. the quantity of the intrument that we possess after the trade
4. The amount of cash available to us for trading the intrument.

In [1]:
import sqlite3
from contextlib import closing

conn = None
with closing(sqlite3.connect("SP500.db")) as conn:
    cs = conn.cursor();
    cs.execute(F"""
    create table if not exists positions (
    time_of_trade text,
    instrument text,
    quantity real,
    cash real,
    primary key (time_of_trade, instrument)
    );
    """);


## Question 2
Choose a lookback time of 120 days and perform backtesting on the prices' from 2021-05-02 to 2021-05-31.

**hint**

1. On each day from 2021-05-02 to 2021-05-31, take the latest 120 days' prices before this date and calibrate a GBM model.
2. Use the GBM model to forecast the price in 10 days:
   $$S_{t + n\Delta t} = S_t \exp \left[
   \left(
   \mu - \frac{\sigma^2}{2}
   \right)n \Delta t
   + W_{t + n \Delta t} - W_t
   \right]$$
The price forecast is
$$
\mathbb E S_{t + n\Delta t} = S_t \exp \left(
   \mu n \Delta t
\right)
$$
In this case, $n=10, \Delta t = 1/250$. Use your code from question 3 of exercise 3 to estimate the confidence interval of the forecast.

3. The relative return of buying the instrument on day 0 and selling it on day 10 is approximately normally distributed with mean $(\mu - \frac{\sigma^2}{2}) n \Delta t$ and standard deviation $\sigma \sqrt{n \Delta t}$. Calculate the 95% expected shortfall $ES_\alpha$ in 10 days:
$$ES_\alpha = -m + s \frac{\phi(\Phi^{-1}(\alpha))}{1 - \alpha}$$
where $\alpha = 0.95$ and
$$m = \left(\mu - \frac{\sigma^2}{2} \right) n \Delta t$$
$$s = \sigma \sqrt{n \Delta t}$$
$\phi$ and $\Phi^{-1}$ are the density and the quantile functions of the standard normal distribution, respectively. You can calculate them in Python using scipy.stats.norm.pdf and scipy.stats.norm.ppf, respectively.

In [2]:
import numpy as np
from scipy.stats import norm
mu = 0.0511
sigma = 0.1440
n = 10
Dt = 1.0/250
m = (mu - sigma**2/2) * n * Dt
s = sigma * np.sqrt(n * Dt)

alpha = 0.95
ES = -m + s * norm.pdf(norm.ppf(alpha))/(1 - alpha)
print(F"The 95% expected shortfall is {ES}")



The 95% expected shortfall is 0.05777684885621387


4. Devise your buy/sell signal based on the forecasted price and its confidence interval. For example, if your forecast suggests that you can be 80% sure that the price in 10 days will be higher than the current price, buy the instrument.


In [ ]:
def position_size(which_day, forecast):
    cs.execute(F"""
    select quantity, cash from positions
    where instrument = 'SP500'
    and time_of_trade < '{which_day}'
    order by time_of_trade desc
    limit 1;
    """)
    qty, cash = cs.fetchall()[0]
    cs.execute(F"""
    select price from prices
    where theday <= '{which_day}'
    order by theday desc
    limit 1;
    """);
    price = cs.fetchall()[0][0]
    capital = cash + qty * price
    if price < forecast['interval'][0]:
        return round(capital/price)
    elif price > forecast['interval'][1]:
        return -round(capital/price)
    else:
        return 0


5. Decide your risk appetite in terms of the *Expected Shortfall*. For example, if you decide that your risk appetite is 95% expected shortfall being 5%, then the ratio of capital that you should use for the trade is $\frac{0.05}{ES_{0.95}}$. If this figure is larger than 1, you may consider using leverage.


**Solution**: modify the position_size function as follows:

In [ ]:
def position_size(which_day, forecast, ES):
    cs.execute(F"""
    select quantity, cash from positions
    where instrument = 'SP500'
    and time_of_trade < '{which_day}'
    order by time_of_trade desc
    limit 1;
    """)
    qty, cash = cs.fetchall()[0]
    cs.execute(F"""
    select price from prices
    where theday <= '{which_day}'
    order by theday desc
    limit 1;
    """);
    price = cs.fetchall()[0][0]
    capital = cash + qty * price
    exposure = capital * 0.05/ES
    if price < forecast['interval'][0]:
        return round(exposure/price)
    elif price > forecast['interval'][1]:
        return -round(exposure/price)
    else:
        return 0


6. Follow your buy/sell signals through the said time period. Store in the database the number of the instruments that you hold and the amount of your cash reserve after each trade.

In [1]:
def analyse(which_day):
    cs.execute(F"""
    select price from prices where theday <= '{which_day}'
    order by theday desc limit 120;
    """)
    P = np.flipud(np.asarray(cs.fetchall())).flatten();
    model = GBM();
    Dt = 1.0/252;
    model.calibrate(P, Dt);
    confidence = 0.1
    n = 10
    T = n * Dt;
    forecast = model.forecast(P[-1], T, confidence);

    m = (model.mu - model.sigma**2/2) * n * Dt
    s = model.sigma * np.sqrt(n * Dt)

    alpha = 0.95
    ES = -m + s * norm.pdf(norm.ppf(alpha))/(1 - alpha)
    return position_size(which_day, forecast, ES);
    
def main(begin_on):
    cs.execute(F"select theday from prices where theday >= '{begin_on}';")
    days = [d[0] for d in cs.fetchall()]
    asset = {
        'old': np.nan,
        'new': np.nan
    };
    cash = {
        'old': np.nan,
        'new': np.nan
    };
    cs.execute("delete from positions where time_of_trade > '2020-01-01';");
    for d in days:
        asset['new'] = analyse(d)
        cs.execute(F"""
        select quantity, cash from positions
        where time_of_trade < '{d}'
        order by time_of_trade desc
        limit 1;
        """);
        asset['old'], cash['old'] = cs.fetchall()[0];
        cs.execute(F"""
        select price from prices
        where theday <= '{d}'
        order by theday desc
        limit 1;
        """);
        latest = cs.fetchall()[0][0]
        trade_size = round(asset['new']) - round(asset['old']);
        if trade_size != 0:
            cash['new'] = cash['old'] - trade_size * latest;
            cs.execute(F"""
            insert into positions values
            ('{d}', 'SP500', {round(asset['new'])}, {cash['new']});
            """);
        conn.commit();

